# Vegetation Indices (VIs) Calculation Using Harmonized Landsat and Sentinel-2 (HLS) Imagery

Using the Harmonized Landsat and Sentinel-2 (HLS) surface reflectance dataset<sup> [1]</sup>, the team is developing canopy chlorophyll and Gross Primary Production (GPP) equations based on Vegetation Indices (VIs). The team was working with extracted HLS data and VIs, and is now applying the GPP equations to the HLS imagery. The data for this project is currently available under the NASA Center for Climate Simulation (NCCS) ADAPT system, primarily located in the following directories:

```bash
L30: /att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30
S30: /att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.S30/S30
YEARS: 2015-2019
```

This notebook is an enhancement of existing MATLAB scripts in order to calculate VIs from extracted spectral values. The idea is to apply the calculated indices to the images and make spatial/geo-referenced maps of the VIs. The team would like to scale the indices 0-1 and apply the canopy chlorophyll and GPP equations to the VI images (scaled and not scaled). This notebook is arquitected to apply to L30 and S30 imagery, and it will require slight modifications in order to apply to additional datasets. For additional information on the bands available via the HLS dataset, feel free to visit <https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/>.

**Author:** Jordan A. Caraballo-Vega - NASA GSFC, <jordan.a.caraballo-vega@nasa.gov> <br/>
**Release Date:** 07/23/2021 <br/>
**Version:** 1.0.9 <br/>

## 1. Usage and installation requirements

## 1.1 Creating a conda environment (One time only)

In order to run this notebook you will need a conda environment with all dependencies installed. ADAPT provides a built-in environment from the JupyterHub interface that is only missing a couple of packages that can be installed on the fly. In order to get started quickly, follow the next steps:

1. Login to adaptlogin.nccs.nasa.gov
2. Load the Anaconda module

```bash
module load anaconda3
```

3. Install new environment or clone the existing environment

```bash
conda config --add channels conda-forge
conda config --set channel_priority strict
conda create -y -n hls-vis-gpp rioxarray cupy cudatoolkit=11.2 dask-cuda cudnn cutensor nccl ipykernel ipywidgets matplotlib geopandas iteration_utilities
```

or

```bash
conda create --name hls-vis --clone /att/nobackup/jacaraba/.conda/envs/hls-vis
```

Now you are ready to move on to JupyterHub.

## 1.2 Login to ADAPT JupyterHub

To leverage NCCS ADAPT resources, you will need to login to ADAPT JupyterHub. The steps are outlined below.

1. Login to the NCCS JupyterHub <https://www-proxy-dev.nccs.nasa.gov/jupyterhub-prism/>.
2. Open this notebook via the file/upload method.
3. Select kernel, in this case "hls-vis".
4. Start working on your notebook.

## 2. Define global variables for the HLS dataset


## 2.1 Import Python Libraries

In this section we include all Python libraries required to execute the code below. There are no external code dependencies besides the packages installed under section 1.1.

In [27]:
import os
from datetime import datetime
from glob import glob
import numpy as np
import xarray as xr
import rioxarray as xrx
import cupy as cp
import matplotlib.pyplot as plt
from matplotlib import colors
import ipywidgets as widgets
import multiprocessing as mp

print("Number of processors: ", mp.cpu_count())

Number of processors:  40


## 2.1 Define Global Variables

In this section we define global variables utilized across the entire notebook. The variables together with their description are listed below. In this section the user will define the directory where the data resides.

- **DATA_PATH (string):** define the directory where data resides, year directories should be located under this path followed by .hdf files. An example of this would be: /att/nobackup/user/L30, where /att/nobackup/user/L30/yyyy/imagery.hdf exists.
- **OUTPUT_PATH (string):** define the directory where output data will be stored, year directories will be auto-generated under this path followed by .hdf files. An example of this would be: /att/nobackup/user/output/L30, where /att/nobackup/user/output/L30/yyyy/imagery_vis.tif will be created.

In this section the user will need to define the DATA_PATH and OUTPUT_PATH variables.

In [28]:
# User should modify DATA_PATH and OUTPUT_PATH accordingly
DATA_PATH = '/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30'
OUTPUT_PATH = '/att/gpfsfs/briskfs01/ppl/jacaraba/testing-gpp-gpu-01-mask/L30'
#DATA_PATH = '/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.S30/S30'
#OUTPUT_PATH = '/att/gpfsfs/briskfs01/ppl/jacaraba/testing-gpp-gpu-01/S30'
NUM_PROCESSES = 10



- **MASK_OPTIONS (dict):** The HLS dataset provides a set of masks that can be retrieved from the QA band of the imagery. The variable MASK_OPTIONS lets the user define which masks are available for selection. This variable will then be used in the checkbox menu to determine which masks will be calculated and applied.
- **VIS_OPTIONS_S30 (dict):** A list of VIs is available through the provided MATLAB script. Different sets of VIs can be calculated based on the selected satellite. This variable defines the available VIs from the Sentinel imagery.
- **VIS_OPTIONS_L30 (dict):** Similar to VIS_OPTIONS_S30, this varible defines the available VIs from the Landsat imagery.

No input from the user is required in this section, unless bands in the imagery change, or new VIs are introduced in this project for calculation.

In [29]:
# DO NOT MODIFY THIS SECTION UNLESS YOU HAVE MORE MASKS OR INDICES TO INCLUDE.
# IPYWIDGETS REQUIRES A DICTIONARY THAT INCLUDES BOTH THE NAME AND LABEL.

# MASK_OPTIONS: DICTIONARY WITH AVAILABLE MASKS
MASK_OPTIONS = {
    'ADJCLOUD':    'ADJCLOUD',
    'AQ':          'AQ',
    'CIRRUS':      'CIRRUS',
    'CLOUD':       'CLOUD',
    'CLOUDSHADOW': 'CLOUDSHADOW',
    'SNOW':        'SNOW',
    'WATER':       'WATER'
}

# MASK_BITS: DICTIONARY WITH MASK BITS RANGE
MASK_BITS = {
    'ADJCLOUD':    [5, 6],
    'AQ':          [0, 2],
    'CIRRUS':      [7, 8],
    'CLOUD':       [6, 7],
    'CLOUDSHADOW': [4, 5],
    'SNOW':        [3, 4],
    'WATER':       [2, 3]
}

# BANDS: DICTIONARY WITH BANDS FROM IMAGERY FOR GPU TRANSFER
BANDS = {
    'L30': [
        'band01', 'band11', 'QA', 'band02', 'band03', 'band04', 'band05', 'band06', 'band07', 'band09', 'band10'
    ],
    'S30': [
        'B01', 'B09', 'B10', 'B11', 'B12', 'QA', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A'
    ],
}

In [30]:
# DO NOT MODIFY THIS SECTION UNLESS YOU HAVE MORE MASKS OR INDICES TO INCLUDE.
# IPYWIDGETS REQUIRES A DICTIONARY THAT INCLUDES BOTH THE NAME AND LABEL.

# VIS_OPTIONS_S30: DICTIONARY WITH AVAILABLE VIS FOR S30
VIS_OPTIONS_S30 = {
    'SR':          'SR',
    'NDVI':        'NDVI',
    'SAVI':        'SAVI',
    'MSAVI':       'MSAVI',
    'OSAVI':       'OSAVI',
    'EVI':         'EVI',
    'TVI':         'TVI',
    'MTVI1':       'MTVI1',
    'MTVI2':       'MTVI2',
    'CVI':         'CVI',
    'GNDVI':       'GNDVI',
    'CIG':         'CIG',
    'NGRDI':       'NGRDI',
    'GLI':         'GLI',
    'VARI':        'VARI',
    'FCVI':        'FCVI',
    'FCVI_VIS':    'FCVI_VIS',
    'NDREI':       'NDREI',
    'CIRE':        'CIRE',
    'MTCI':        'MTCI',
    'MCARI':       'MCARI',
    'TCARI':       'TCARI',
    'TCI':         'TCI',
    'TCARI_OSAVI': 'TCARI_OSAVI',
    'MCARI_MTVI2': 'MCARI_MTVI2',
    'TGI':         'TGI',
    'NDVI705':     'NDVI705',
    'NDVI740':     'NDVI740',
    'CI705':       'CI705',
    'CI740':       'CI740',
    'CCCI':        'CCCI',
    'RE1RE2':      'RE1RE2',
    'REIP3':       'REIP3',
}

# VIS_OPTIONS_L30: DICTIONARY WITH AVAILABLE VIS FOR L30
VIS_OPTIONS_L30 = {
    'SR':          'SR',
    'NDVI':        'NDVI',
    'SAVI':        'SAVI',
    'MSAVI':       'MSAVI',
    'OSAVI':       'OSAVI',
    'EVI':         'EVI',
    'TVI':         'TVI',
    'MTVI1':       'MTVI1',
    'MTVI2':       'MTVI2',
    'CVI':         'CVI',
    'GNDVI':       'GNDVI',
    'CIG':         'CIG',
    'NGRDI':       'NGRDI',
    'GLI':         'GLI',
    'VARI':        'VARI',
    'FCVI':        'FCVI',
    'FCVI_VIS':    'FCVI_VIS',
}

## 3. Define variables to extract VIs from the HLS imagery

## 3.1 Widget Functions

In this section we define widget functions to assist in the execution of this notebook. These functions take care of the visual implementations of elements that will then be used to finalize Mask and VIs calculations. No input from the user is required in this section.

In [31]:
def checkbox_menu(data: dict) -> list:
    """
    Define dynamic widgets for checkbox menu.
    Input:
        data (dict): dictionary with key and label to initiate checkbox menu
    Return: list of selected objects
    """
    names = list()
    checkbox_objects = list()
    for key in data:
        if key == 'AQ': # AQ mask is [1,2,3]
            checkbox_objects.append(widgets.Checkbox(value=False, description=key))
        else:
            checkbox_objects.append(widgets.Checkbox(value=True, description=key))
        names.append(key)

    # generate dictionary of all arguments in the checkbox menu
    arg_dict = {names[i]: checkbox for i, checkbox in enumerate(checkbox_objects)}

    # divide the options in 4 columns, and generate horizontal grid
    chunk = int(round(len(checkbox_objects)/3))
    ui = widgets.HBox(
        [
            widgets.VBox(checkbox_objects[i:i+chunk]) for i in range(0, len(checkbox_objects), chunk)
        ]
    )

    # dynamically allocate values to variable
    selected_data = []
    def select_data(**kwargs):
        selected_data.clear()
        for key in kwargs:
            if kwargs[key] is True:
                selected_data.append(key)
        print(selected_data)

    out = widgets.interactive_output(select_data, arg_dict)
    display(ui)
    return selected_data

def get_years(data_path: str, year_options: dict = {}) -> dict:
    """
    Retrieve dataset available years.
    Input:
        data_path (str): string with directory where data resides.
    Return: dict of available years
    """
    for y in glob(f'{DATA_PATH}/*'):
        year = y.split('/')[-1]
        year_options[year] = year
    return sorted(year_options)

## 3.2 Specify data directory, masks and VIs

This notebook allows the user to select between Landsat (L30) and Sentinel-2 (S30) imagery in order to calculate the respective Masks. In this section the user will define the years in question. In addition, the user will select the desired Masks and VIs for calculation.

Select between the 3 available checkbox menus for year, masks, and VIs.

In [32]:
SATELLITE = DATA_PATH.split('/')[-1]  # selecting between L30 and S30 based on the last directory of DATA_PATH, no user intervention
YEAR_OPTIONS = get_years(DATA_PATH)  # retrieving dataset available years, no user intervention

print(f"Selecting satellite {SATELLITE} and DATA_PATH {DATA_PATH}")

Selecting satellite L30 and DATA_PATH /att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30


- **YEARS (list string):** define the years that will be processed by this script
- **MASKS (list string):** define the masks that will be processed by this script
- **VIS (list string):** define the VIS that will be calculated by this script

In [33]:
# years checkbox menu
print("Selected Years:")
YEARS = checkbox_menu(YEAR_OPTIONS)

# mask checkbox menu
print("Selected Masks:")
MASKS = checkbox_menu(MASK_OPTIONS)

# vis checkbox menu
print(f"Selected VIs for {SATELLITE}:")
VIS = checkbox_menu(VIS_OPTIONS_L30) if SATELLITE == 'L30' else checkbox_menu(VIS_OPTIONS_S30)

Selected Years:


Selected Masks:


Selected VIs for L30:


In [34]:
# print a small summary
print(f"Calculating {len(YEARS)} years, {len(MASKS)} masks, and {len(VIS)} VIs.")

Calculating 5 years, 6 masks, and 17 VIs.


## 4. Start Mask and VIs Calculations

In this section, data files are retrieved and indices are calculated. Make sure to specify the variables in the previous section to allow for a seamless calculation.

## 4.1 Get all imagery files

In this section we mine through the DATA_PATH to retrive all filenames that will be processed. No input from the user is required in this section.

In [35]:
def get_filenames(data_path: str, years_var: list = ['ALL'], f_ext: str = '.hdf'):
    """
    Retrieve filenames to calculate indices from.
    Input:
        data_path (str): string with directory where data resides.
        years_var (str list): list of years to work with.
        f_ext (str): imagery filename extensions.
    Return: list of filenames to process
    """
    filenames = list()
    if 'ALL' in years_var:  # iterate over all years under the main data path
        filenames = glob(f'{data_path}/**/*{f_ext}', recursive=True) + filenames
    else: # iterate over the years specified by the user
        for y in years_var:
            filenames = glob(f'{data_path}/{y}/*{f_ext}', recursive=True) + filenames
    assert len(filenames) > 0, f"No files were found in {data_path}."
    return filenames

In [36]:
filenames = get_filenames(data_path=DATA_PATH, years_var=YEARS)
print(f"Processing {len(filenames)} files...")

Processing 222 files...


## 4.2 Define function to decode QA mask band

Here we need to decode the QA Mask Band. Examples of how to do it are outlined below. No input from the user is required in this section.

In [37]:
def get_binary(z: int, width: int = 8) -> str:
    """
    Retrieve binary representation.
    Input:
        z (int): string with value to decode
        width (int): int identity (HLS is 8 bits)
    Return: binary representation for a single value in str format
    """
    if z < 0:
        return '0' * 8
    else:
        return np.binary_repr(z, width=width)
    
def get_mask(z: str, mid: str, width: int = 8, start_bit: int = 0, end_bit: int = 1) -> int:
    """
    Retrieve mask from binary representation.
    Input:
        z (str): string with binary representation
        width (int): int identity (HLS is 8 bits)
        start_bit (int): position of bit in string (starts with 0)
        end_bit (int): position of bit in string (starts with 1)
    Return: return pixel mask value
    """
    if z == '0' * width:
        return 0
    else:
        if mid == 'AQ': # bin2dec(code(1:2))
            return int(z[start_bit:end_bit],2)
        else: # 1 - str2num(code(7))
            return 1 - int(z[start_bit:end_bit])   

# vectorize get_binary and get_mask functions
v_get_binary = np.vectorize(get_binary, doc='Vectorized `get_binary_repr`')
v_get_mask = np.vectorize(get_mask, doc='Vectorized `get_mask`')

## 4.3 Define function to apply QA masks

Here we apply the QA Mask. Examples of how to do it are outlined below. No input from the user is required in this section.

In [38]:
def calculate_mask(qa, mask_id, mask_bits, width=8):
    start_bit, end_bit = mask_bits[mask_id][0], mask_bits[mask_id][1]
    return {
        mask_id: v_get_mask(qa, mask_id, width=width, start_bit=start_bit, end_bit=end_bit)
    }

## 4.4 Define function to apply S30 VIs

Here we calculate S30 VIs. Examples of how to do it are outlined below. No input from the user is required in this section.

In [39]:
def apply_vis_s30(ds, VIS_list: list = ['FCVI_VIS']):
    """
    Calculate S30 VIs.
    S30 Bands - B01, B09, B10, B11, B12, QA, B02, B03, B04, B05, B06, B07, B08, B8A
    Input:
        ds (xarray): array with dataset and all bands
        VIS_list (list): list of selected VIs to apply
    Return: xarray with VIs applied
    """
    with np.errstate(all='ignore'):

        if 'SR' in VIS_list:
            # b8a_s30_s./b04_s30_s
            ds['SR'] = ds['B8A'] / ds['B04']

        if 'NDVI' in VIS_list:
            # (b8a_s30_s-b04_s30_s)./(b8a_s30_s+b04_s30_s)
            ds['NDVI'] = (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'])

        if 'SAVI' in VIS_list:
            # (1+0.5).*(b8a_s30_s-b04_s30_s)./((b8a_s30_s+b04_s30_s+0.5))
            ds['SAVI'] = (1 + 0.5) * (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'] + 0.5)
            
        if 'MSAVI' in VIS_list:
            # 0.5.*(2 * b8a_s30_s + 1 - sqrt( (2 * b8a_s30_s + 1).^2-8.*(b8a_s30_s-b04_s30_s) ) )
            ds['MSAVI'] = 0.5 * (2 * ds['B8A'] + 1 - np.sqrt( (2 * ds['B8A'] + 1)**2e-8 * (ds['B8A'] - ds['B04'])))
            
        if 'OSAVI' in VIS_list:
            # (1+0.16).*(b8a_s30_s-b04_s30_s)./(b8a_s30_s+b04_s30_s+0.16)
            ds['OSAVI'] = (1 + 0.16) * (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'] + 0.16)
            
        if 'EVI' in VIS_list:
            # 2.5.*(b8a_s30_s-b04_s30_s)./(b8a_s30_s+6*b04_s30_s-7.5*b02_s30_s+1)
            ds['EVI'] = 2.5 * (ds['B8A'] - ds['B04']) / (ds['B8A'] + 6 * ds['B04'] - 7.5 * ds['B02'] + 1)

        if 'TVI' in VIS_list:
            # 0.5.*(120.*(b8a_s30_s-b03_s30_s)-200.*(b04_s30_s-b03_s30_s))
            ds['TVI'] = 0.5 * (120 * (ds['B8A'] - ds['B03']) - 200 * (ds['B04'] - ds['B03']))

        if 'MTVI1' in VIS_list:
            # 1.2*(1.2*(b8a_s30_s-b03_s30_s)-2.5*(b04_s30_s-b03_s30_s))
            ds['MTVI1'] = 1.2 * (1.2 * (ds['B8A'] - ds['B03']) - 2.5 * (ds['B04'] - ds['B03']))

        if 'MTVI2' in VIS_list:
            # 1.5*((1.2*(b8a_s30_s-b03_s30_s)-2.5.*(b04_s30_s-b03_s30_s))./sqrt((2*b8a_s30_s+1).^2-(6.*b8a_s30_s-5*sqrt(b04_s30_s))-0.5))
            ds['MTVI2'] = 1.5 * ((1.2 * (ds['B8A'] - ds['B03']) - 2.5 * (ds['B04'] - ds['B03'])) / np.sqrt((2*ds['B8A']+1)**2 - (6 * ds['B8A'] - 5 * np.sqrt(ds['B04'])) - 0.5))

        if 'CVI' in VIS_list:
            # ((b8a_s30_s).*(b04_s30_s))./((b03_s30_s).^2)
            ds['CVI'] = ((ds['B8A']) * (ds['B04'])) / ((ds['B03'])**2)
            
        if 'GNDVI' in VIS_list:
            # (b8a_s30_s-b03_s30_s)./(b8a_s30_s+b03_s30_s)
            ds['GNDVI'] = (ds['B8A'] - ds['B03']) / (ds['B8A'] + ds['B03'])
            
        if 'CIG' in VIS_list:
            # (b8a_s30_s./b04_s30_s)-1
            ds['CIG'] = (ds['B8A'] / ds['B04']) - 1.0
            
        if 'NGRDI' in VIS_list:
            # (b03_s30_s-b04_s30_s)./(b03_s30_s+b04_s30_s)
            ds['NGRDI'] = (ds['B03'] - ds['B04']) / (ds['B03'] + ds['B04'])
            
        if 'GLI' in VIS_list:
            # (2*b03_s30_s-b04_s30_s-b02_s30_s)./(2*b03_s30_s+b04_s30_s+b02_s30_s)
            ds['GLI'] = (2 * ds['B03'] - ds['B04'] - ds['B02']) / (2 * ds['B03'] + ds['B04'] + ds['B02'])
            
        if 'VARI' in VIS_list:
            # (b03_s30_s-b04_s30_s)./(b03_s30_s+b04_s30_s-b02_s30_s)
            ds['VARI'] = (ds['B03'] - ds['B04']) / (ds['B03'] + ds['B04'] - ds['B02'])
            
        if 'FCVI' in VIS_list:
            # (b8a_s30_s-(b02_s30_s+b03_s30_s+b04_s30_s)./3)
            ds['FCVI'] = (ds['B8A'] - (ds['B02'] + ds['B03'] + ds['B04']) / 3.0)

        if 'FCVI_VIS' in VIS_list:
            # (b8a_s30_s-(b02_s30_s+b03_s30_s+b04_s30_s)./3)./((b02_s30_s+b03_s30_s+b04_s30_s)./3)
            ds['FCVI_VIS'] = (ds['B8A'] - (ds['B02'] + ds['B03'] + ds['B04']) / 3.0) / ((ds['B02'] + ds['B03'] + ds['B04']) / 3.0)

        if 'NDREI' in VIS_list:
            # (b8a_s30_s-b06_s30_s)./(b8a_s30_s+b06_s30_s);
            ds['NDREI'] = (ds['B8A'] - ds['B06']) / (ds['B8A'] + ds['B06'])

        if 'CIRE' in VIS_list:
            # (b8a_s30_s./b06_s30_s)-1, note: same formula as CI740
            ds['CIRE'] = (ds['B8A'] / ds['B06']) - 1.0

        if 'MTCI' in VIS_list:
            # (b06_s30_s-b05_s30_s)./(b05_s30_s-b04_s30_s);
            ds['MTCI'] = (ds['B06'] - ds['B05']) / (ds['B05'] - ds['B04'])
            
        if 'MCARI' in VIS_list:
            # ((b05_s30_s-b04_s30_s)-0.2.*(b05_s30_s-b03_s30_s))./((b06_s30_s)./(b04_s30_s));
            ds['MCARI'] = ((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B06'] / ds['B04'])

        if 'TCARI' in VIS_list:
            # 3*(((b05_s30_s-b04_s30_s)-0.2*(b05_s30_s-b03_s30_s))./(b05_s30_s./b04_s30_s));
            ds['TCARI'] = 3 * (((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B05'] / ds['B04']))
            
        if 'TCI' in VIS_list:
            # 1.2*(b05_s30_s-b03_s30_s)-1.5.*(b04_s30_s-b03_s30_s).*sqrt(b05_s30_s./b04_s30_s);
            ds['TCI'] = 1.2 * (ds['B05'] - ds['B03']) - 1.5 * (ds['B04'] - ds['B03']) * np.sqrt(ds['B05'] / ds['B04'])
            
        if 'TCARI_OSAVI' in VIS_list:
            # TCARI_S30./OSAVI_S30; dependent on TCARI and OSAVI
            ds['TCARI'] = 3 * (((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B05'] / ds['B04']))
            ds['OSAVI'] = (1 + 0.16) * (ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04'] + 0.16)
            ds['TCARI_OSAVI'] = ds['TCARI'] / ds['OSAVI']

        if 'MCARI_MTVI2' in VIS_list:
            # MCARI_S30./MTVI2_S30; dependent on MCARI and MTVI2
            ds['MCARI'] = ((ds['B05'] - ds['B04']) - 0.2 * (ds['B05'] - ds['B03'])) / (ds['B06'] / ds['B04'])
            ds['MTVI2'] = 1.5 * ((1.2 * (ds['B8A'] - ds['B03']) - 2.5 * (ds['B04'] - ds['B03'])) / np.sqrt((2*ds['B8A']+1)**2 - (6 * ds['B8A'] - 5 * np.sqrt(ds['B04'])) - 0.5))
            ds['MCARI_MTVI2'] = ds['MCARI'] / ds['MTVI2']
            
        if 'TGI' in VIS_list:
            # ((b04_s30_s-(b02_s30_s).*(b04_s30_s-b03_s30_s))-(b04_s30_s-(b03_s30_s).*(b04_s30_s-b02_s30_s))).*(-0.5);
            ds['TGI'] = ((ds['B04'] - (ds['B02']) * (ds['B04'] - ds['B03'])) - (ds['B04'] - (ds['B03']) * (ds['B04'] - ds['B02']))) * (-0.5)

        if 'NDVI705' in VIS_list:
            # (b8a_s30_s-b05_s30_s)./(b8a_s30_s+b05_s30_s);
            ds['NDVI705'] = (ds['B8A'] - ds['B05']) / (ds['B8A'] + ds['B05'])

        if 'NDVI740' in VIS_list:
            # (b8a_s30_s-b06_s30_s)./(b8a_s30_s+b06_s30_s);
            ds['NDVI740'] = (ds['B8A'] - ds['B06']) / (ds['B8A'] + ds['B06'])

        if 'CI705' in VIS_list:
            # (b8a_s30_s./b05_s30_s)-1
            ds['CI705'] = (ds['B8A'] / ds['B05']) - 1.0

        if 'CI740' in VIS_list:
            # (b8a_s30_s./b06_s30_s)-1
            ds['CI740'] = (ds['B8A'] / ds['B06']) - 1.0

        if 'CCCI' in VIS_list:
            # ((b8a_s30_s-b06_s30_s)./(b8a_s30_s+b06_s30_s))./((b8a_s30_s-b04_s30_s)./(b8a_s30_s+b04_s30_s))
            ds['CCCI'] = ((ds['B8A'] - ds['B06']) / (ds['B8A'] + ds['B06'])) / ((ds['B8A'] - ds['B04']) / (ds['B8A'] + ds['B04']))

        if 'RE1RE2' in VIS_list:
            # (b06_s30_s./b05_s30_s)-1;
            ds['RE1RE2'] = (ds['B06'] / ds['B05']) - 1.0

        if 'REIP3' in VIS_list:
            # 705+35.*(((((b04_s30_s+b8a_s30_s)./2)-b05_s30_s))./(b06_s30_s-b05_s30_s));
            ds['REIP3'] = 705 + 35 * (((((ds['B04'] + ds['B8A']) / 2) - ds['B05'])) / (ds['B06'] - ds['B05']))

    return ds.fillna(0)  # fill nan values with 0s

## 4.5 Define function to apply L30 VIs

Here we calculate L30 VIs. Examples of how to do it are outlined below. No input from the user is required in this section.

In [40]:
def apply_vis_l30(ds, VIS_list: list = ['SR']):
    """
    Calculate L30 VIs.
    L30 Bands - band01, band11, QA, band02, band03, band04, band05, band06, band07, band09, band10
    Input:
        ds (xarray): array with dataset and all bands
        VIS_list (list): list of selected VIs to apply
    Return: xarray with VIs applied
    """
    with np.errstate(all='ignore'):

        if 'SR' in VIS_list:
            # SR_L30=b05_l30_s./b04_l30_s;
            ds['SR'] = ds['band05'] / ds['band04']

        if 'NDVI' in VIS_list:
            # NDVI_L30=(b05_l30_s-b04_l30_s)./(b05_l30_s+b04_l30_s);
            ds['NDVI'] = (ds['band05'] - ds['band04']) / (ds['band05'] + ds['band04'])

        if 'SAVI' in VIS_list:
            # SAVI_L30=(1+0.5).*(b05_l30_s-b04_l30_s)./((b05_l30_s+b04_l30_s+0.5)); 
            ds['SAVI'] = (1 + 0.5) * (ds['band05'] - ds['band04']) / ((ds['band05'] + ds['band04'] + 0.5))
        
        if 'MSAVI' in VIS_list:
            # MSAVI_L30=0.5.*(2.*b05_l30_s+1-sqrt((2.*b05_l30_s+1).^2-8.*(b05_l30_s-b04_l30_s)));
            ds['MSAVI'] = 0.5 * (2 * ds['band05'] + 1 - np.sqrt((2 * ds['band05'] + 1)**2e-8 * (ds['band05'] - ds['band04'])))

        if 'OSAVI' in VIS_list:
            # OSAVI_L30=(1+0.16).*(b05_l30_s-b04_l30_s)./(b05_l30_s+b04_l30_s+0.16);
            ds['OSAVI'] = (1 + 0.16) * (ds['band05'] - ds['band04']) / (ds['band05'] + ds['band04'] + 0.16)

        if 'EVI' in VIS_list:
            # EVI_L30=2.5.*(b05_l30_s-b04_l30_s)./(b05_l30_s+6.*(b04_l30_s)-7.5.*(b02_l30_s)+1);
            ds['EVI'] = 2.5 * (ds['band05'] - ds['band04']) / (ds['band05'] + 6 * (ds['band04']) - 7.5 * (ds['band02']) + 1)

        if 'TVI' in VIS_list:
            # TVI_L30=0.5.*(120.*(b05_l30_s-b03_l30_s)-200.*(b04_l30_s-b03_l30_s));
            ds['TVI'] = 0.5 * (120 * (ds['band05'] - ds['band03']) - 200 * (ds['band04'] - ds['band03']))

        if 'MTVI1' in VIS_list:
            # MTVI1_L30=1.2*(1.2*(b05_l30_s-b03_l30_s)-2.5*(b04_l30_s-b03_l30_s));
            ds['MTVI1'] = 1.2 * (1.2 * (ds['band05'] - ds['band03']) - 2.5 * (ds['band04'] - ds['band03']))

        # is the exponent after the negative? or is that a substraction?
        if 'MTVI2' in VIS_list:
            # MTVI2_L30=1.5*((1.2*(b05_l30_s-b03_l30_s)-2.5.*(b04_l30_s-b03_l30_s))./sqrt((2*b05_l30_s+1).^2-(6.*b05_l30_s-5*sqrt(b04_l30_s))-0.5));
            ds['MTVI2'] = 1.5 * ((1.2 * (ds['band05'] - ds['band03']) - 2.5 * (ds['band04'] - ds['band03'])) / np.sqrt((2 * ds['band05'] + 1)**2 - (6 * ds['band05']- 5 * np.sqrt(ds['band04'])) - 0.5))
        
        if 'CVI' in VIS_list:
            # CVI_L30=(b05_l30_s).*((b04_l30_s)./((b03_l30_s).^2));
            ds['CVI'] = (ds['band05']) * ((ds['band04']) / ((ds['band03'])**2))

        if 'GNDVI' in VIS_list:
            # GNDVI_L30=(b05_l30_s-b03_l30_s)./(b05_l30_s+b03_l30_s);
            ds['GNDVI'] = (ds['band05'] - ds['band03']) / (ds['band05'] + ds['band03'])

        if 'CIG' in VIS_list:
            # CIG_L30=((b05_l30_s)./(b04_l30_s))-1;
            ds['CIG'] = ((ds['band05']) / (ds['band04'])) - 1.0

        if 'NGRDI' in VIS_list:
            # NGRDI_L30=(b03_l30_s-b04_l30_s)./(b03_l30_s+b04_l30_s);
            ds['NGRDI'] = (ds['band03'] - ds['band04']) / (ds['band03'] + ds['band04'])

        if 'GLI' in VIS_list:
            # GLI_L30=(2.*(b03_l30_s)-(b04_l30_s+b02_l30_s))./(2.*(b03_l30_s)+b04_l30_s+b02_l30_s);
            ds['GLI'] = (2 * (ds['band03']) - (ds['band04'] + ds['band02'])) / (2 * (ds['band03']) + ds['band04'] + ds['band02'])

        if 'VARI' in VIS_list:
            # VARI_L30=(b03_l30_s-b04_l30_s)./(b03_l30_s+b04_l30_s-b02_l30_s);
            ds['VARI'] = (ds['band03'] - ds['band04']) / (ds['band03'] + ds['band04'] - ds['band02'])

        if 'FCVI' in VIS_list:
            # FCVI_L30=(b05_l30_s-(b02_l30_s+b03_l30_s+b04_l30_s)./3);
            ds['FCVI'] = (ds['band05'] - (ds['band02'] + ds['band03'] + ds['band04']) / 3.0)

        if 'FCVI_VIS' in VIS_list:
            # FCVI_VIS_L30=(b05_l30_s-(b02_l30_s+b03_l30_s+b04_l30_s)./3)./((b02_l30_s+b03_l30_s+b04_l30_s)./3);
            ds['FCVI_VIS'] = (ds['band05'] - (ds['band02'] + ds['band03'] + ds['band04']) / 3) / ((ds['band02'] + ds['band03'] + ds['band04']) / 3)

    return ds.fillna(0)  # fill nan values with 0s

## 4.6 Iterate over each file and output GeoTIF files

Here we define the function to iterate over each file and process both mask and VIs. Each file is taking around ~3 minutes to process, saving into disk takes some time. 

The process has been slightly parallelized, but can be improved even more. Two additional things can be parallelized here: process many files at the same time or parallelize functions to calculate indices.

In [76]:
def _xarray_to_cupy_(ds, band: str):
    return xr.DataArray(cp.asarray(ds[band]), dims=["band", "y", "x"])

def _xarray_to_numpy_(ds, band: str):
    return xr.DataArray(cp.asnumpy(ds[band]), dims=["band", "y", "x"])

def process_filename(f: str, vis: list, masks: list, output_path: str):
    """
    Process each HDF HLS file.
    Input:
        f (str): filename
        vis (list): list of selected VIs to apply
        masks (list): list of selected masks to apply
        output_path (str): directory to store files
    Return: xarray with VIs applied
    """
    try:
        
        # create output directory if not present
        output_dir = os.path.join(output_path, f.split('/')[-2])
        os.makedirs(output_dir, exist_ok=True)

        # define output filename
        output_file = os.path.join(output_dir, f.split('/')[-1][:-4] + '.gpp.vis.tif')

        if not os.path.exists(output_file):

            # open the dataset from the hdf imagery
            hls_ds = xrx.open_rasterio(f)

            # decode qa_mask for the generation of external masks
            qa_mask = v_get_binary(hls_ds['QA'].values)

            # reorder bands since rasterio reads them in numbered order, move to GPU
            hls_ds_tmp, hls_ds = hls_ds, hls_ds.drop(BANDS[SATELLITE])

            for b in sorted(BANDS[SATELLITE]):
                hls_ds[b] = _xarray_to_cupy_(ds=hls_ds_tmp, band=b)
            hls_ds = (hls_ds.drop('QA') / 10000.0).astype(cp.float32)

            # parallelize mask calculation process
            pool = mp.Pool(processes=NUM_PROCESSES)
            mask_processes = [pool.apply_async(calculate_mask, args=(qa_mask, x, MASK_BITS, 8)) for x in masks]

            # retrieve and apply required masks
            final_mask = cp.ones((hls_ds['y'].shape[0], hls_ds['x'].shape[0]), dtype=cp.int8)            
            for p in mask_processes:
                final_mask = final_mask * cp.asarray(list(p.get().values())[0])
            final_mask = hls_ds[BANDS[SATELLITE][0]].copy(data=final_mask).astype(bool)

            # calculate indices
            hls_ds = apply_vis_l30(hls_ds, vis) if SATELLITE == 'L30' else apply_vis_s30(hls_ds, vis)            
            hls_ds = xr.where(final_mask, hls_ds, -9999.0).astype('float32')
            
            # need to drop some indices before output to local disk?
            # dropind =  ['band11', 'band04', 'band05', 'band06', 'band07', 'band09', 'band10']
            # hls_ds = hls_ds.drop(dropind)
            
            hls_ds.rio.write_crs("EPSG:32618", inplace=True)
            hls_ds.isel(band=0).rio.to_raster(output_file, compress='LZW')

            return output_file

        else:
            print(f"{output_file} exists, skipping.")
            return None
    
    except KeyError:
        print(f'{f} does not have a QA band available, skipping the calculation of VIs. Check module installation.')
        return None


In [ ]:
%%time
# Serial approach 
# 5 files
# CPU times: user 2min 18s, sys: 31.8 s, total: 2min 50s
# Wall time: 4min 33s
for f in filenames:
    print(f, datetime.now().strftime("%H:%M:%S"))    
    process_filename(f, vis=VIS, masks=MASKS, output_path=OUTPUT_PATH)    

/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2019/HLS.L30.T18SUJ.2019041.v1.4.hdf 11:41:56
/att/gpfsfs/briskfs01/ppl/jacaraba/testing-gpp-gpu-01-mask/L30/2019/HLS.L30.T18SUJ.2019041.v1.4.gpp.vis.tif exists, skipping.
/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2019/HLS.L30.T18SUJ.2019192.v1.4.hdf 11:41:56
/att/gpfsfs/briskfs01/ppl/jacaraba/testing-gpp-gpu-01-mask/L30/2019/HLS.L30.T18SUJ.2019192.v1.4.gpp.vis.tif exists, skipping.
/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2019/HLS.L30.T18SUJ.2019160.v1.4.hdf 11:41:56
/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2019/HLS.L30.T18SUJ.2019249.v1.4.hdf 11:43:03
/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2019/HLS.L30.T18SUJ.2019217.v1.4.hdf 11:44:42
/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2019/HLS.L30.T18SUJ.2019064.v1.4.hdf 11:45:35
/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/201

In [ ]:
# %%time
# from iteration_utilities import grouper
# for f1, f2 in list(grouper(filenames[:5], 2, fillvalue=None)):
#    if f1:
#        with cp.cuda.Device(0):
#            process_filename(f1, vis=VIS, masks=MASKS, output_path=OUTPUT_PATH)
#    if f2:
#        with cp.cuda.Device(1):
#            process_filename(f2, vis=VIS, masks=MASKS, output_path=OUTPUT_PATH)

In [ ]:
# %%time
# parallel - n number of files based on N processes - Skip this section since dealing with GPU
# future implementation will include nested processes to bypass daemonic features
# jobs = []
# for f in filenames:
#    p = mp.Process(target=process_filename, args=(f, VIS, MASKS, OUTPUT_PATH))
#    jobs.append(p)
#    p.start()
#    
# for j in jobs:
#    j.join()

# AssertionError: daemonic processes are not allowed to have children
# parallelize mask calculation process
# pool_filenames = mp.Pool(processes=NUM_PROCESSES)
# filenames_init = [pool_filenames.apply_async(process_filename, args=(f, VIS, MASKS, OUTPUT_PATH)) for f in filenames]

# retrieve and apply required masks
# filenames_final = [p.get() for p in filenames_init]

## 5. Visual Validation (Optional)

In this section we visualize some bands to confirm the output of the notebook include the original dataset, calculated masks, and calculated VIs. The user may specify any files here for further analysis.

In [ ]:
# ORIGINAL_FILENAME = '/att/gpfsfs/briskfs01/ppl/pentchev/OPE3_HLS/HLS.GSFC.18SUJ.L30/L30/2015/HLS.L30.T18SUJ.2015149.v1.4.hdf'
# VIS_FILENAME = '/att/gpfsfs/briskfs01/ppl/jacaraba/testing-gpp/L30/2015/HLS.L30.T18SUJ.2015149.v1.4.gpp.vis.tif'

In [ ]:
# visualize original filename
# original_ds = xrx.open_rasterio(ORIGINAL_FILENAME)
# band01, band02, band03 = original_ds['band01'].values, original_ds['band02'].values, original_ds['band03'].values
# original_vis = np.concatenate((band01, band02, band03), axis=0)
# original_vis = np.clip(original_vis, 0, 10000)
# original_vis = np.moveaxis(original_vis, 0, -1) / 10000.0
# plt.imshow(original_vis)
# plt.show()

In [ ]:
# visualize vis filename
# vis_ds = xrx.open_rasterio(VIS_FILENAME)
# vis_ds.attrs['long_name']

In [ ]:
# vis_CIG_band = vis_ds[10, :, :].values
# vis_CIG_band = np.moveaxis(vis_CIG_band, 0, -1)
# plt.imshow(vis_CIG_band)
# plt.show()

In [ ]:
# from matplotlib import colors
# cmap = colors.ListedColormap(['r','b'])
# az = np.squeeze(hls_ds['SAVI'])
# plt.imshow(az)#, cmap=cmap)
# plt.show()